"Pinging" typically refers to sending a small data packet from one computer or device to another on a network, usually over the Internet. This is done to check the availability and responsiveness of the target device or server. The process involves sending an ICMP (Internet Control Message Protocol) Echo Request packet to the target and waiting for an ICMP Echo Reply. The time taken for the round trip (known as latency or ping time) is measured, which helps assess network performance and troubleshoot connectivity issues. Pinging is commonly used in diagnostics, network monitoring, and online gaming to measure and optimize network performance.

In [ ]:
from pythonping import ping
target = 'ftp.nz.debian.org'
ping(target, verbose=False, count=20)

Now we will try to use scapy to do the same thing. We will create several ICMP echo request packet, send it to our target server and wait for the response. Let's try to understand how scapy works.

First we will understand how to create packets by stacking protocol layers

In [ ]:
from scapy.all import *

#default IP layer
IP().show()

#default ICMP layer
ICMP().show()

#default UDP layer
UDP().show()

#default TCP layer
TCP().show()

#create a simple ICMP echo request packet with IP destination 'ftp.nz.debian.org'. You can stack layers with the '/' operand.
packet = IP(dst = target) / ICMP()
packet.show()

Now we'll send the packet we have created. The sr() function can be used to send packets and receive answers at the IP layer. 

The sr() function returns two outputs: i) a couple of packets sent and answers received and ii) the unanswered packets. 

```
ans, unans = sr(packet)
```

The structure of the object for answered packets is the following: ans[i][0] stores the i-th sent packet, ans[i][1] the i-th received response 

In [ ]:
ans, unans = sr(packet, verbose = False)
print('Received ', len(ans), 'packet')
print("The Complete Sent and Received Packets are : ")
print(ans.summary())
query = ans[0][0]
answer = ans[0][1]
print("Seperated : ")
print("***** Query (Sent Traffic) : ***** ")
print(query.summary())
print("***** Received Traffic :  *****")
print(answer.summary())

We can also get some timing info from the sent/received packets...





In [ ]:
#print(query.sent_time)
print(answer.time)
rtt = answer.time - query.sent_time
print(rtt*1000) #in ms

Now we can generate our own ping command using Scapy

In [ ]:
import statistics as stat
RTT = []
lost = 0
replied = 0
c = 20
for x in range(c):
  packet = IP(dst = target) / ICMP(seq=x)
  ans, unans = sr(packet, verbose = False)
  if(len(ans)>0):
    RTT.append((ans[0][1].time - ans[0][0].sent_time)*1000) #in ms
    replied=replied+1
  if(len(unans)>0):
    lost+=unans

print('Round Trip Times min/avg/max/std','%.2f' % min(RTT),'%.2f' % (sum(RTT)/len(RTT)), '%.2f' % max(RTT), '%.2f' % stat.stdev(RTT))
print('Packets transmitted/replied/unanswered: ',c, replied, lost)